In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import datetime

# Start spark session
startTime = datetime.datetime.now()

spark = SparkSession \
    .builder\
    .appName("Anime importer") \
    .master("spark://spark-master:7077") \
    .config("spark.driver.memory", "15g") \
    .config("spark.jars", "/extra_jars/neo4j-connector-apache-spark_2.12-4.0.1_for_spark_3.jar") \
    .config("neo4j.url", "bolt://neo4j:7687")\
    .config("neo4j.authentication.type", "basic")\
    .config("neo4j.authentication.basic.username", "neo4j")\
    .config("neo4j.authentication.basic.password", "password")\
    .getOrCreate()

print(datetime.datetime.now()-startTime)

0:00:03.538981


In [ ]:
# Read animes with cypher
startTime = datetime.datetime.now()
print(startTime)

(spark.read.format("org.neo4j.spark.DataSource")
#  .option("url", "bolt://neo4j:7687")
 .option("query", "MATCH (a:Anime) WITH a LIMIT 2 RETURN id(a) as id, a.title as title")
 .load()
 .show())

print(datetime.datetime.now()-startTime)

2021-06-17 15:09:16.439919


In [ ]:
# Read animes with cypher 2
startTime = datetime.datetime.now()

query = """
    CALL gds.graph.create('got-interactions', 'Person', {
      INTERACTS: {
        orientation: 'UNDIRECTED'
      }
    })
    YIELD graphName, nodeCount, relationshipCount, createMillis
    RETURN graphName, nodeCount, relationshipCount, createMillis
"""

query = """
    MATCH (n) WITH n LIMIT 10 
    RETURN n
"""

query = """
    CALL gds.graph.create.cypher(
        '1001r_graph', 
        'MATCH (n) RETURN id(n) AS id', 
        'MATCH (n:User)-[r:WATCHED]->(m:Anime) 
            RETURN id(n) AS source, 
            id(m) AS target, 
            r.my_watched_episodes AS weight 
        LIMIT 1001')
"""

(spark.read.format("org.neo4j.spark.DataSource")
 .option("url", "bolt://neo4j:7687")
 .option("query", query)
 .option("partitions", "1")
 .load()
 .show())

# (spark.read.format("org.neo4j.spark.DataSource")
#  .option("url", "bolt://neo4j:7687")
#  .option("query", "CALL gds.graph.create.cypher(graphName: 'full_graph_spark3', nodeQuery: 'MATCH (a:Anime) WITH a LIMIT 100 RETURN id(a) AS id', relationshipQuery: 'MATCH (n:User)-[r:WATCHED]->(m:Anime) RETURN id(n) AS source, id(m) AS target, r.my_watched_episodes AS weight LIMIT 100', configuration: {validateRelationships:false})")
#  .load()
#  .show())

print(datetime.datetime.now()-startTime)